For more information about `squadv2` metrics click here https://huggingface.co/spaces/evaluate-metric/squad_v2

In [1]:
!pip install datasets --quiet
!pip install evaluate --quiet

In [2]:
from evaluate import load
from transformers import RobertaTokenizer, RobertaForQuestionAnswering
from datasets import load_dataset
import torch, re

In [3]:
model_name = "deepset/roberta-base-squad2"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [43]:
squad_dev = load_dataset("squad_v2", split="train[90:100]")
# squad_dev = load_dataset("squad_v2", split="train+validation") # UNCOMMENT THIS WHEN YOU KNOW RESULTS CAN PRINT
squad_metric = load("squad_v2")

In [44]:
predictions = []
references = []

for i in range(squad_dev.num_rows):
  id = squad_dev[i]['id']
  question = squad_dev[i]["question"]
  context = squad_dev[i]["context"]
  actual_answers = squad_dev[i]["answers"]
  if len(actual_answers) == 0:
    actual_answers = ""

  tokenized_example = tokenizer(question, context, return_tensors="pt", truncation="only_second")

  with torch.no_grad():
    output = model(**tokenized_example)

  start_logits = output.start_logits
  end_logits = output.end_logits

  start_index = torch.argmax(start_logits)
  end_index = torch.argmax(end_logits)

  answer_tokens = tokenized_example['input_ids'][0][start_index:end_index + 1]
  predicted_answer = tokenizer.decode(answer_tokens).strip()
  # predicted_answer = re.sub(r'<s>|</s>', '', predicted_answer)
  no_answer_probability = 0
  if predicted_answer == "<s>":
    predicted_answer = ""
    no_answer_probability = 1

  predictions.append({'id': id, 'prediction_text': predicted_answer, 'no_answer_probability': no_answer_probability})
  references.append({'id': id, 'answers': actual_answers})

  # print(f"Question: {question}")
  # print(f"Actual Answer: {actual_answers}")
  # print(f"Predicted Answer: {predicted_answer}")
  # print()

Question: What event occured after she was publicly criticized?
Actual Answer: {'text': ['boyfriend left her'], 'answer_start': [320]}
Predicted Answer: Her long-standing boyfriend left her

Question: Who supported Beyonce through her depression?
Actual Answer: {'text': ['her mother'], 'answer_start': [714]}
Predicted Answer: her mother

Question: What event caused Beyonce's depression?
Actual Answer: {'text': ['split with Luckett and Rober'], 'answer_start': [194]}
Predicted Answer: split

Question: How long was Beyonce depressed?
Actual Answer: {'text': ['a couple of years'], 'answer_start': [396]}
Predicted Answer: a couple of years

Question: Who helped Beyonce fight her depression the most?
Actual Answer: {'text': ['her mother'], 'answer_start': [714]}
Predicted Answer: her mother

Question: Who replaced Luckett and Roberson in Destiny's Child?
Actual Answer: {'text': ['Farrah Franklin and Michelle Williams.'], 'answer_start': [110]}
Predicted Answer: Farrah Franklin and Michelle 

In [45]:
results = squad_metric.compute(predictions=predictions, references=references)
results

{'exact': 80.0,
 'f1': 90.83333333333334,
 'total': 10,
 'HasAns_exact': 80.0,
 'HasAns_f1': 90.83333333333334,
 'HasAns_total': 10,
 'best_exact': 80.0,
 'best_exact_thresh': 0.0,
 'best_f1': 90.83333333333334,
 'best_f1_thresh': 0.0}